In [1]:
import os, sys
sys.path.append(os.path.abspath("..")) # to be able to import src

In [2]:
from src.utils.logging import select_best_model

In [3]:
res = select_best_model('qm9')

2025/12/08 13:56:45 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/08 13:56:45 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Best overall run ID: 50a90f1034a24848b786d90b23698285
Best VAL_MSE: 0.1553


In [4]:
res

{'run_id': '50a90f1034a24848b786d90b23698285',
 'model_type': 'schnet',
 'model_uri': 'runs:/50a90f1034a24848b786d90b23698285/model'}

thta id corresponds to the tuning id!!!!

In [5]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [6]:


def get_experiment_id_by_name(name):
    exp = client.get_experiment_by_name(name)
    if exp is None:
        raise ValueError(f"Experiment '{name}' not found.")
    return exp.experiment_id

experiment_name = "qm9"
experiment_id = get_experiment_id_by_name(experiment_name)
experiment_id


'1'

In [7]:
import mlflow
import pandas as pd

experiment_id = "1"   # <-- change to your experiment
desired_tag = res["run_id"]  # the value you want to match


df = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string=(
        "attributes.run_name = 'test_evaluation'"
        f"and tags.tuning_run_id = '{desired_tag}'"
    )
)

df


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_r2,metrics.test_mse,metrics.test_mae,metrics.test_ev,metrics.test_rmse,params.model_type,tags.tuning_run_id,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.runName,tags.data_version,tags.dataset
0,f95ff7bbb83a483fa0c567c2ba883a53,1,FINISHED,/home/marcos/Escritorio/AI-prod/QM9-ML/mlruns/...,2025-12-05 23:05:44.974000+00:00,2025-12-05 23:05:45.097000+00:00,0.283612,0.455627,0.51568,0.319306,0.675002,schnet,50a90f1034a24848b786d90b23698285,/home/marcos/Escritorio/AI-prod/QM9-ML/scripts...,marcos,fdc2bdd2ce2a41b386952801490671c76f136447,LOCAL,test_evaluation,v1,test
1,244d456cca344c168a8fd217fdb77190,1,FINISHED,/home/marcos/Escritorio/AI-prod/QM9-ML/mlruns/...,2025-12-05 17:44:22.932000+00:00,2025-12-05 17:44:23.674000+00:00,0.283612,0.455627,0.51568,0.319306,0.675002,schnet,50a90f1034a24848b786d90b23698285,/home/marcos/Escritorio/AI-prod/QM9-ML/scripts...,marcos,24f627e1a12353718ee16ea0bcaa266eec3cbaa2,LOCAL,test_evaluation,v1,test
2,031df118ece140eebb7876be34ce60cd,1,FINISHED,/home/marcos/Escritorio/AI-prod/QM9-ML/mlruns/...,2025-12-04 22:28:34.569000+00:00,2025-12-04 22:28:34.692000+00:00,0.283612,0.455627,0.51568,0.319306,0.675002,schnet,50a90f1034a24848b786d90b23698285,/home/marcos/Escritorio/AI-prod/QM9-ML/scripts...,marcos,0221ee5ee7d6a7cddf9a739c29908dee729984e2,LOCAL,test_evaluation,v1,test


In [8]:
run_id = df.iloc[0]["run_id"]

In [9]:
run_id


run = client.get_run(run_id)

run_info = run.info            # start/end time, status, user, etc.
run_params = run.data.params   # dict
run_metrics = run.data.metrics # dict of last logged values
run_tags = run.data.tags       # dict

#print("INFO:", run_info)
print("PARAMS:", run_params)
print("METRICS:", run_metrics)
#print("TAGS:", run_tags)


PARAMS: {'model_type': 'schnet'}
METRICS: {'test_mse': 0.45562735199928284, 'test_rmse': 0.6750017404556274, 'test_mae': 0.5156804919242859, 'test_r2': 0.28361213207244873, 'test_ev': 0.31930607557296753}


# for a code snippet

In [ ]:
# load results from test
# get results from the prev. given run_id

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
run_id = res["run_id"]
run_id = "336b9fcac6a144ae88006d286e5439c3"

# ---- 1. RUN METADATA ----
run = client.get_run(run_id)

run_info = run.info            # start/end time, status, user, etc.
run_params = run.data.params   # dict
run_metrics = run.data.metrics # dict of last logged values
run_tags = run.data.tags       # dict

print("INFO:", run_info)
print("PARAMS:", run_params)
print("METRICS:", run_metrics)
print("TAGS:", run_tags)

"""
list and download aryifacts

artifacts = client.list_artifacts(run_id)
artifact_paths = [a.path for a in artifacts]
artifact_paths

local_dir = client.download_artifacts(run_id, "")
print("Downloaded to:", local_dir)


"""


In [ ]:
run_params

In [ ]:
run_metrics

In [ ]:
# load best model for inference

In [ ]:
"""
def get_run_data(run_id):
    client = MlflowClient()
    run = client.get_run(run_id)

    data = {
        "info": run.info,
        "params": run.data.params,
        "metrics": run.data.metrics,
        "tags": run.data.tags,
        "artifacts": {}
    }

    # download artifacts
    for a in client.list_artifacts(run_id):
        local_path = client.download_artifacts(run_id, a.path)
        data["artifacts"][a.path] = local_path

    return data

all_data = get_run_data("417c54e0be4f4b1f813036d6194144b2")
all_data
"""